In [1]:
import pandas as pd
import numpy as np

In [2]:
## Definimos path

pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'

In [3]:
## Cargo y ordeno los indicadores

ratio_actividad = pd.read_csv(pathdata + 'data_output/indicadores/01_ratio_actividad.csv', index_col=0)
ratio_formalidad = pd.read_csv(pathdata + 'data_output/indicadores/02_ratio_formalidad.csv', index_col=0)
ratio_jornada_laboral = pd.read_csv(pathdata + 'data_output/indicadores/03_ratio_jornada_laboral.csv', index_col=0)
ratio_ingreso_salarial = pd.read_csv(pathdata + 'data_output/indicadores/04_ratio_ingreso_salarial.csv', index_col=0)

ratio_jornada_no_paga = pd.read_csv(pathdata + 'data_output/indicadores/05_ratio_jornada_no_paga.csv', index_col=0)
ratio_inactivos_con_ingreso = pd.read_csv(pathdata + 'data_output/indicadores/06_ratio_inactivos_con_ingreso.csv', index_col=0)
ratio_hogares_no_pobres_jefatura = pd.read_csv(pathdata + 'data_output/indicadores/07_ratio_hogares_no-pobres_jefatura.csv', index_col=0)


ratio_actividad = ratio_actividad.sort_index()
ratio_formalidad = ratio_formalidad.sort_index()
ratio_jornada_laboral = ratio_jornada_laboral.sort_index()
ratio_ingreso_salarial = ratio_ingreso_salarial.sort_index()

ratio_jornada_no_paga = ratio_jornada_no_paga.sort_index()
ratio_inactivos_con_ingreso = ratio_inactivos_con_ingreso.sort_index()
ratio_hogares_no_pobres_jefatura = ratio_hogares_no_pobres_jefatura.sort_index()


In [4]:
## Chequeo que todos los indicadores tengan ordenados los indices

variables = [ratio_actividad, ratio_formalidad, ratio_jornada_laboral, ratio_ingreso_salarial, \
             ratio_jornada_no_paga, ratio_inactivos_con_ingreso, ratio_hogares_no_pobres_jefatura]

indices_ordenados = True

for i in range(len(variables) - 1):
    if not variables[i].index.equals(variables[i + 1].index):
        indices_ordenados = False
        break

print("Los índices están ordenados de la misma manera:", indices_ordenados)

Los índices están ordenados de la misma manera: True


In [5]:
## Trunco y calculo el ranking de los indicadores

variables = [ratio_actividad, ratio_formalidad, ratio_jornada_laboral, ratio_ingreso_salarial, \
             ratio_jornada_no_paga, ratio_inactivos_con_ingreso, ratio_hogares_no_pobres_jefatura]

columnas = ['Actividad', 'Formalidad', 'Horas promedio remuneradas', 'Ingreso laboral promedio', \
            'Tiempo TNR', 'Dependencia', 'No Pobreza']

for columna, variable in zip(columnas,variables):
    variable[columna] = np.where(variable[columna] > 100, 100, variable[columna])
    variable['Ranking'] = variable[columna].rank(ascending=False, method='first')


In [6]:
## Agrego componentes e indice

componente_insercion = (ratio_actividad['Actividad'] + \
                        ratio_formalidad['Formalidad'] + \
                        ratio_jornada_laboral['Horas promedio remuneradas'] + \
                        ratio_ingreso_salarial['Ingreso laboral promedio'])/4

componente_insercion = componente_insercion.to_frame()
componente_insercion.columns = ['Insercion laboral']
componente_insercion['Ranking'] = componente_insercion['Insercion laboral'].rank(ascending=False, method='first')

componente_uso_del_tiempo_y_oportunidades = (ratio_jornada_no_paga['Tiempo TNR'] + \
                                ratio_inactivos_con_ingreso['Dependencia'] + \
                                ratio_hogares_no_pobres_jefatura['No Pobreza'])/3

componente_uso_del_tiempo_y_oportunidades = componente_uso_del_tiempo_y_oportunidades.to_frame()
componente_uso_del_tiempo_y_oportunidades.columns = ['Uso del tiempo y oportunidades']
componente_uso_del_tiempo_y_oportunidades['Ranking'] = componente_uso_del_tiempo_y_oportunidades['Uso del tiempo y oportunidades'].rank(ascending=False, method='first')

indice_gti = np.sqrt(componente_uso_del_tiempo_y_oportunidades['Uso del tiempo y oportunidades'] * componente_insercion['Insercion laboral']).to_frame()
indice_gti.columns = ['Indice GTI']
indice_gti['Ranking'] = indice_gti['Indice GTI'].rank(ascending=False, method='first')

In [9]:
## Genero tabla resumen de medias, desvio, maximo, minimo y rango para cada variable

labels_variables_todas = ['Actividad', 'Formalidad', 'Horas promedio remuneradas', 'Ingreso laboral promedio',\
                  'Tiempo TNR', 'Dependencia', 'No Pobreza',\
                  'Insercion laboral', 'Uso del tiempo y oportunidades', 'Indice GTI']

columnas = ['Media', 'Desvio estandar', 'Minimo', 'Maximo', 'Rango']

variables_todas = [ratio_actividad, ratio_formalidad, ratio_jornada_laboral, ratio_ingreso_salarial, \
                        ratio_jornada_no_paga, ratio_inactivos_con_ingreso, ratio_hogares_no_pobres_jefatura, \
                            componente_insercion, componente_uso_del_tiempo_y_oportunidades, indice_gti]

df_tabla_resumen = pd.DataFrame(index=labels_variables_todas, columns=columnas)

for i, idf in enumerate(variables_todas):
    icolumna = labels_variables_todas[i]
    imedia = idf[icolumna].mean()
    istd = idf[icolumna].std()
    imin = idf[icolumna].min() 
    imax = idf[icolumna].max()
    irango = imax - imin
    df_tabla_resumen.iloc[i,0] = imedia
    df_tabla_resumen.iloc[i,1] = istd
    df_tabla_resumen.iloc[i,2] = imin
    df_tabla_resumen.iloc[i,3] = imax
    df_tabla_resumen.iloc[i,4] = irango

df_tabla_resumen = df_tabla_resumen.astype('float').round(2)

In [11]:
## Genero tabla output con todos los indicadores, componentes e indice general.

columnas = ['Actividad', 'Formalidad', 'Horas promedio remuneradas', 'Ingreso laboral promedio', 'Tiempo TNR', 'Dependencia', 'No Pobreza']

provincias = indice_gti.index.to_list()
provincia_repetidos = [value for value in provincias for _ in range(7)]

componentes = ['Insercion laboral'] * 4 + ['Uso del tiempo y oportunidades'] * 3
componentes_repetidos = componentes*24

indicadores = ['Actividad', 'Formalidad', 'Horas trabajadas', 'Ingreso salarial',
                  'Tiempo trabajo no remunerado', 'Dependencia economica', 'No-Pobreza']

data = {
    'Provincia': provincia_repetidos,
    'Componente': componentes_repetidos,
    'Indicador': indicadores*24
}

df = pd.DataFrame(data)

multiindex = pd.MultiIndex.from_frame(df[['Provincia', 'Componente', 'Indicador']])
df.set_index(multiindex, inplace=True)
df.drop(columns=['Provincia', 'Componente', 'Indicador'], inplace=True)

df[['Mujer', 'Varon', 'Ratio', 'N pob varon', 'N pob mujer', 'N pob tot', 'N varon', 'N mujer', 'CV', 'Error', 'Ranking indicador', 'Componentes', 'Ranking componente', 'Indice', 'Ranking Indice']] = pd.NaT

for indicador, columna, variable in zip(indicadores, columnas, variables):
    if indicador == 'Tiempo trabajo no remunerado':
        values = variable[['Mujer', 'Varon', columna, 'Ranking']].values
        df.loc[df.index.get_level_values('Indicador') == indicador, ['Mujer', 'Varon', 'Ratio', 'Ranking indicador']] = values

    else:    
        values = variable[['Mujer', 'Varon', columna, 'N_pob_v', 'N_pob_m', 'N_pob_tot', 'N_v', 'N_m', 'ER', 'ME', 'Ranking']].values
        df.loc[df.index.get_level_values('Indicador') == indicador, ['Mujer', 'Varon', 'Ratio', 'N pob varon', 'N pob mujer', 'N pob tot', 'N varon', 'N mujer', 'CV', 'Error', 'Ranking indicador']] = values

values = componente_insercion.values
df.loc[df.index.get_level_values('Indicador') == 'Actividad', ['Componentes', 'Ranking componente']] = values

values = componente_uso_del_tiempo_y_oportunidades.values
df.loc[df.index.get_level_values('Indicador') == 'Tiempo trabajo no remunerado', ['Componentes', 'Ranking componente']] = values

values = indice_gti.values
df.loc[df.index.get_level_values('Indicador') == 'Actividad', ['Indice', 'Ranking Indice']] = values

In [13]:
## Exporto archivos .csv

df.to_csv(pathdata + 'data_output/tablas/01_indicadores_principales.csv')
componente_insercion.to_csv(pathdata + 'data_output/componentes_e_indice/01_insercion_laboral.csv')
componente_uso_del_tiempo_y_oportunidades.to_csv(pathdata + 'data_output/componentes_e_indice/02_uso_del_tiempo_y_oportunidades.csv')
indice_gti.to_csv(pathdata + 'data_output/componentes_e_indice/01_indice_GTI.csv')